In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf

import tensorflow_addons as tfa
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from keras.utils import pad_sequences

In [2]:
df=pd.read_csv('input9.csv')

In [3]:
df.head()

,comment,class
0,FIXME formatters are not thread safe,DEFECT
1,"XXX Jon Skeet The comment ""if it hasn't bee...",DEFECT
2,"I hate to admit it, but we don't know what h...",DEFECT
3,Just a note StarTeam has a status for NEW wh...,DEFECT
4,the generated classes must not be added in t...,DEFECT


In [4]:
df['class'].value_counts()

WITHOUT_CLASSIFICATION    58204
DESIGN                     2703
IMPLEMENTATION              757
DEFECT                      472
TEST                         85
DOCUMENTATION                54
Name: class, dtype: int64

In [5]:
df['class']=df['class'].apply(lambda x: 0 if x=='WITHOUT_CLASSIFICATION' else 1)

In [6]:
df['class'].value_counts()

0    58204
1     4071
Name: class, dtype: int64

In [8]:
X = df['comment']
Y = df['class']
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [9]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20, stratify = Y, random_state=42)

In [10]:
max_words = 10000
max_len = 250
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [11]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(128)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [12]:
model = RNN()
model.summary()
opt = tfa.optimizers.AdaBelief(learning_rate=5e-3)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 250)]             0         
                                                                 
 embedding (Embedding)       (None, 250, 50)           500000    
                                                                 
 lstm (LSTM)                 (None, 128)               91648     
                                                                 
 FC1 (Dense)                 (None, 256)               33024     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [13]:
#model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=True,
                monitor='val_loss',
                mode='min',
                save_best_only=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001)

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=50,validation_split=0.25, callbacks=[model_checkpoint_callback,reduce_lr])

model.load_weights(checkpoint_filepath)

Epoch 1/50
292/292 [==============================] - 14s 38ms/step - loss: 0.1656 - accuracy: 0.9566 - val_loss: 0.0774 - val_accuracy: 0.9756 - lr: 0.0050
Epoch 2/50
292/292 [==============================] - 10s 36ms/step - loss: 0.0775 - accuracy: 0.9752 - val_loss: 0.0691 - val_accuracy: 0.9772 - lr: 0.0050
Epoch 3/50
292/292 [==============================] - 10s 36ms/step - loss: 0.0640 - accuracy: 0.9797 - val_loss: 0.0665 - val_accuracy: 0.9807 - lr: 0.0050
Epoch 4/50
292/292 [==============================] - 10s 36ms/step - loss: 0.0771 - accuracy: 0.9760 - val_loss: 0.1856 - val_accuracy: 0.9379 - lr: 0.0050
Epoch 5/50
292/292 [==============================] - 10s 36ms/step - loss: 0.0713 - accuracy: 0.9765 - val_loss: 0.0617 - val_accuracy: 0.9809 - lr: 0.0050
Epoch 6/50
292/292 [==============================] - 10s 35ms/step - loss: 0.0457 - accuracy: 0.9847 - val_loss: 0.0657 - val_accuracy: 0.9823 - lr: 0.0050
Epoch 7/50
292/292 [==============================] - 10s 

In [14]:
model.save("LSTM_imbalanced.h5")

In [15]:
accr = model.evaluate(test_sequences_matrix,Y_test)

390/390 [==============================] - 4s 11ms/step - loss: 0.0697 - accuracy: 0.9793


In [16]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.070
  Accuracy: 0.979


In [17]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred)

390/390 [==============================] - 4s 11ms/step
[[5.6039367e-04]
 [8.9281693e-06]
 [1.7107379e-04]
 ...
 [1.3418356e-01]
 [6.0818519e-04]
 [4.6752337e-01]]


In [18]:
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]

In [19]:
print(classification_report(Y_test, y_pred, digits=6))

              precision    recall  f1-score   support

           0   0.984507  0.993471  0.988969     11641
           1   0.892655  0.776413  0.830486       814

    accuracy                       0.979285     12455
   macro avg   0.938581  0.884942  0.909727     12455
weighted avg   0.978504  0.979285  0.978611     12455

